<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/LLM_GPT2_QA_Finetune_Instruct_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import transformers
print(transformers.__version__)

4.47.1


Finetuning:

Instruct

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from datasets import Dataset

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token

# Prepare 5 QA samples for instruction tuning
qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']} Answer:"
    target_text = example["answer"]

    # Tokenize inputs and labels
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=256)
    labels = tokenizer(target_text, truncation=True, padding="max_length", max_length=256)

    # Set labels to ignore padding tokens
    inputs["labels"] = [label if label != tokenizer.pad_token_id else -100 for label in labels["input_ids"]]
    return inputs

# Convert samples to Hugging Face dataset and preprocess
dataset = Dataset.from_list(qa_samples)
tokenized_dataset = dataset.map(preprocess_data)

# Split dataset into train and eval
# split_dataset = tokenized_dataset.train_test_split(test_size=0.2)

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-qa-finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=2,
    learning_rate=5e-5,
    num_train_epochs=20,
    logging_dir="./logs",
    logging_steps=1,
    save_total_limit=2,
    weight_decay=0.01,
    push_to_hub=False,
    fp16=True,  # Enable mixed precision training for faster training
    report_to=[],  # Disable W&B or any reporting
)

# Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# Start training
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./gpt2-qa-finetuned")
tokenizer.save_pretrained("./gpt2-qa-finetuned")

# Load the fine-tuned model for inference
model = GPT2LMHeadModel.from_pretrained("./gpt2-qa-finetuned").to("cuda")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-qa-finetuned")

# Inference function
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.pad_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-dd796ea81604>:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,11.723400,11.080130
2,9.574200,11.080130
3,8.447900,8.626542
4,8.393600,7.327123
5,8.900800,6.092465
6,5.819800,5.137457
7,6.751600,4.043090
8,3.397500,3.167568
9,4.653500,2.622004
10,2.771700,2.354168


Q: What is the capital of France?
A: . Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris


#Inference: Greedy Search

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
model_name = "./gpt2-qa-finetuned"  # Change to your model path
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

# Ensure padding token is set
tokenizer.pad_token = tokenizer.eos_token

def greedy_generate_answer(question, max_length=50):
    """
    Generate an answer using greedy search.

    Args:
        question (str): The input question.
        max_length (int): Maximum length of the generated sequence.

    Returns:
        str: The generated answer.
    """
    model.eval()
    device = model.device

    # Prepare input with the instruction-based format
    input_text = f"Question: {question} Answer:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Initialize generated token IDs with input
    generated_ids = input_ids.clone()

    with torch.no_grad():
        for _ in range(max_length):
            # Get the model's output logits
            outputs = model(input_ids=generated_ids)
            logits = outputs.logits

            # Select the token with the highest probability (greedy search)
            next_token_id = torch.argmax(logits[:, -1, :], dim=-1).unsqueeze(0)

            # Append the new token to the sequence
            generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)

            # Stop generating if the EOS token is reached
            if next_token_id.item() == tokenizer.eos_token_id:
                break

    # Decode the generated token IDs to text
    answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example usage
question = "What is the capital of France?"
answer = greedy_generate_answer(question)
print(f"Q: {question}\nA: {answer}")


Q: What is the capital of France?
A: . Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris


#Inference: Beam Search

In [ ]:
import torch
import torch.nn.functional as F
import time

def generate_answer(question, model, tokenizer, max_length=50, num_beams=5, device="cuda"):
    """
    Generate an answer using beam search for the fine-tuned GPT-2 model.

    Args:
        question (str): The input question.
        model (GPT2LMHeadModel): The fine-tuned model.
        tokenizer (GPT2Tokenizer): The tokenizer.
        max_length (int): Maximum length of the generated text.
        num_beams (int): Number of beams for beam search.
        device (str): The device to run the model on ('cuda' or 'cpu').

    Returns:
        str: The generated answer.
    """
    model.to(device)
    model.eval()

    # Prepare input text
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True)

    # Extract input_ids and attention_mask
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Initialize beams
    beams = [(input_ids, attention_mask, 0.0)]  # List of (sequence, attention_mask, score)
    completed_sequences = []

    # Beam search loop
    for _ in range(max_length):
        new_beams = []

        for seq, mask, score in beams:
            # Stop expanding if sequence ends with EOS token
            if seq[0, -1] == tokenizer.eos_token_id:
                completed_sequences.append((seq, score))  # Append sequence and score
                continue

            # Forward pass
            with torch.no_grad():
                outputs = model(seq, attention_mask=mask)
                logits = outputs.logits[:, -1, :]  # Logits of the last token
                probs = F.log_softmax(logits, dim=-1)  # Convert to log probabilities

            # Get top-k tokens and their log probabilities
            top_k_probs, top_k_tokens = torch.topk(probs, num_beams, dim=-1)

            # Expand each beam
            for prob, token in zip(top_k_probs[0], top_k_tokens[0]):
                new_seq = torch.cat([seq, token.unsqueeze(0).unsqueeze(0)], dim=1)  # Append token
                new_mask = torch.cat([mask, torch.ones((1, 1), device=device)], dim=1)  # Update attention mask
                new_score = score + prob.item()  # Accumulate log probability as float
                new_beams.append((new_seq, new_mask, new_score))

        # Sort new beams by score and keep top-k
        new_beams = sorted(new_beams, key=lambda x: x[2], reverse=True)[:num_beams]
        beams = new_beams

        # Break if all beams end with EOS token
        if all(seq[0, -1] == tokenizer.eos_token_id for seq, _, _ in beams):
            break

    # Add remaining beams to completed sequences
    completed_sequences.extend([(seq, float(score)) for seq, _, score in beams])

    # Select the sequence with the highest score
    best_sequence, _ = max(completed_sequences, key=lambda x: x[1])

    # Decode the tokens to text
    answer = tokenizer.decode(best_sequence[0], skip_special_tokens=True)
    return answer


# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./gpt2-qa-finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-qa-finetuned")

# Test the generate_answer function
question = "What is the capital of France?"
device = "cuda" if torch.cuda.is_available() else "cpu"
st = time.time()
answer = generate_answer(question, model, tokenizer, max_length=50, num_beams=5, device=device)
en = time.time()
print('total time:', en-st)
print(answer)

total time: 4.90141487121582
Question: What is the capital of France?
Answer:. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris. Paris
